In [1]:
import numpy as np # linear algebra

In [2]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [3]:
import re # regular expression

## Introduction

#### Our goal is to clean, explore and prepare data for analyzing. This is the dataframe which we should work on.

In [7]:
df = pd.read_csv ("C:/Users/User/Downloads/Portfolio_data/Uncleaned_DS_jobs.csv", index_col=0)

df.head()

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
index,,,,,,,,,,,,,,
0,Sr Data Scientist,$137K-$171K (Glassdoor est.),Description\n\nThe Senior Data Scientist is re...,3.1,Healthfirst\n3.1,"New York, NY","New York, NY",1001 to 5000 employees,1993,Nonprofit Organization,Insurance Carriers,Insurance,Unknown / Non-Applicable,"EmblemHealth, UnitedHealth Group, Aetna"
1,Data Scientist,$137K-$171K (Glassdoor est.),"Secure our Nation, Ignite your Future\n\nJoin ...",4.2,ManTech\n4.2,"Chantilly, VA","Herndon, VA",5001 to 10000 employees,1968,Company - Public,Research & Development,Business Services,$1 to $2 billion (USD),-1
2,Data Scientist,$137K-$171K (Glassdoor est.),Overview\n\n\nAnalysis Group is one of the lar...,3.8,Analysis Group\n3.8,"Boston, MA","Boston, MA",1001 to 5000 employees,1981,Private Practice / Firm,Consulting,Business Services,$100 to $500 million (USD),-1
3,Data Scientist,$137K-$171K (Glassdoor est.),JOB DESCRIPTION:\n\nDo you have a passion for ...,3.5,INFICON\n3.5,"Newton, MA","Bad Ragaz, Switzerland",501 to 1000 employees,2000,Company - Public,Electrical & Electronic Manufacturing,Manufacturing,$100 to $500 million (USD),"MKS Instruments, Pfeiffer Vacuum, Agilent Tech..."
4,Data Scientist,$137K-$171K (Glassdoor est.),Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions\n2.9,"New York, NY","New York, NY",51 to 200 employees,1998,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,"Commerce Signals, Cardlytics, Yodlee"


##### Let us first look at the columns.

In [8]:
df.columns

Index(['Job Title', 'Salary Estimate', 'Job Description', 'Rating',
       'Company Name', 'Location', 'Headquarters', 'Size', 'Founded',
       'Type of ownership', 'Industry', 'Sector', 'Revenue', 'Competitors'],
      dtype='object')

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 672 entries, 0 to 671
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Job Title          672 non-null    object 
 1   Salary Estimate    672 non-null    object 
 2   Job Description    672 non-null    object 
 3   Rating             672 non-null    float64
 4   Company Name       672 non-null    object 
 5   Location           672 non-null    object 
 6   Headquarters       672 non-null    object 
 7   Size               672 non-null    object 
 8   Founded            672 non-null    int64  
 9   Type of ownership  672 non-null    object 
 10  Industry           672 non-null    object 
 11  Sector             672 non-null    object 
 12  Revenue            672 non-null    object 
 13  Competitors        672 non-null    object 
dtypes: float64(1), int64(1), object(12)
memory usage: 78.8+ KB


##### The company column is a mix with rating. let's show the company name alone. But first, the company column

In [9]:
df['Company Name'].head()

index
0           Healthfirst\n3.1
1               ManTech\n4.2
2        Analysis Group\n3.8
3               INFICON\n3.5
4    Affinity Solutions\n2.9
Name: Company Name, dtype: object

In [10]:
df['Company Name'] = df['Company Name'].map(lambda x: x.split('\n')[0])

In [12]:
df['Company Name'].head()

index
0           Healthfirst
1               ManTech
2        Analysis Group
3               INFICON
4    Affinity Solutions
Name: Company Name, dtype: object

##### let's change '-1' and 'unknown' values in every column to 'na' and '0' in integer or float data types

In [14]:
df.iloc[90:120]['Competitors']

index
90            Roche, GlaxoSmithKline, Novartis
91                                          -1
92                                          -1
93                                          -1
94                                          -1
95                   Square, PayPal, H&R Block
96                                          -1
97                                          -1
98                Nielsen, Zappi, SurveyMonkey
99                         The World Bank, IMF
100                                         -1
101                                         -1
102                                         -1
103                                         -1
104                  Square, PayPal, H&R Block
105                                         -1
106                                         -1
107                                         -1
108                                         -1
109    Booz Allen Hamilton, CACI International
110                                         -1
111    

In [15]:
string_cols = ['Job Title', 'Salary Estimate', 'Job Description', 'Company Name', 'Location', 'Headquarters', 'Size', 'Type of ownership', 'Industry', 'Sector', 'Revenue', 'Competitors']

df[string_cols] = df[string_cols].replace('-1', 'na')
df[string_cols] = df[string_cols].replace('unknown', 'na')

number_cols = ['Rating', 'Founded']

df[number_cols] = df[number_cols].replace(-1, 0)

##### While fixing it we will extract minimal and maximal estimation of price to another columns and calculate average price.

In [16]:
df['Salary Estimate'].head()

index
0    $137K-$171K (Glassdoor est.)
1    $137K-$171K (Glassdoor est.)
2    $137K-$171K (Glassdoor est.)
3    $137K-$171K (Glassdoor est.)
4    $137K-$171K (Glassdoor est.)
Name: Salary Estimate, dtype: object

In [17]:
pattern = r'\$(\d+)K-\$(\d+)K'

df[['min_salary', 'max_salary']] = df['Salary Estimate'].str.extract(pattern).astype(int)

df['avg_salary'] = df.apply(lambda row: int((row['min_salary'] + row['max_salary']) / 2), axis=1)

df['Salary Estimate'] = df['min_salary'].astype(str) + '-' + df['max_salary'].astype(str)

##### And now it is organized!

In [18]:
df[['Salary Estimate', 'min_salary', 'max_salary', 'avg_salary']].head()

,Salary Estimate,min_salary,max_salary,avg_salary
index,,,,
0,137-171,137,171,154
1,137-171,137,171,154
2,137-171,137,171,154
3,137-171,137,171,154
4,137-171,137,171,154


##### Let's calculate the company's age using 'Founded' column

In [20]:
df['company_age'] = df['Founded'].map(lambda x: 2024 - x if x > 0 else 0)

In [21]:
df['company_age'].head()

index
0    31
1    56
2    43
3    24
4    26
Name: company_age, dtype: int64

##### Look at this two columns, 'Location' columns describes where you will work, in office in certain State in USA, 'Headquarters' columns describes where is headquarter of company. If there is a same informations in these two columns it means you will work in headquarter of company, otherwise not, we will create new columns named 'same_state'.

In [22]:
df[['Location', 'Headquarters']].head()

,Location,Headquarters
index,,
0,"New York, NY","New York, NY"
1,"Chantilly, VA","Herndon, VA"
2,"Boston, MA","Boston, MA"
3,"Newton, MA","Bad Ragaz, Switzerland"
4,"New York, NY","New York, NY"


In [23]:
df['same_state'] = df.apply(lambda row: 1 if row['Location'] == row['Headquarters'] else 0, axis=1)

In [24]:
df[['Location', 'Headquarters', 'same_state']].head()

,Location,Headquarters,same_state
index,,,
0,"New York, NY","New York, NY",1
1,"Chantilly, VA","Herndon, VA",0
2,"Boston, MA","Boston, MA",1
3,"Newton, MA","Bad Ragaz, Switzerland",0
4,"New York, NY","New York, NY",1


##### As you see there is whole bunch of information in 'Job Description' column, we can not read all these one by one. these very popular technologies used in Data Science:
* Python
* Excel
* Hadoop
* Spark
* AWS
* Tableau
* Big Data

If employer requires some of these technologies these technologies must be written in 'Job Description' column, we will create 7 boolean (0/1) columns which refer to these 7 technologies.

In [25]:
df['python']   = df['Job Description'].map(lambda x: 1 if 'python'   in x.lower() else 0)
df['excel']    = df['Job Description'].map(lambda x: 1 if 'excel'    in x.lower() else 0)
df['hadoop']   = df['Job Description'].map(lambda x: 1 if 'hadoop'   in x.lower() else 0)
df['spark']    = df['Job Description'].map(lambda x: 1 if 'spark'    in x.lower() else 0)
df['aws']      = df['Job Description'].map(lambda x: 1 if 'aws'      in x.lower() else 0)
df['tableau']  = df['Job Description'].map(lambda x: 1 if 'tableau'  in x.lower() else 0)
df['big_data'] = df['Job Description'].map(lambda x: 1 if 'big data' in x.lower() else 0)

In [26]:
df[['python', 'excel', 'hadoop', 'spark', 'aws', 'tableau', 'big_data']]

,python,excel,hadoop,spark,aws,tableau,big_data
index,,,,,,,
0,0,0,0,0,1,0,0
1,0,0,1,0,0,0,1
2,1,1,0,0,1,0,0
3,1,1,0,0,1,0,0
4,1,1,0,0,0,0,0
...,...,...,...,...,...,...,...
667,1,1,1,0,0,1,1
668,0,0,0,0,0,0,0
669,1,1,1,0,0,1,0


In [27]:
df.head()

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,...,avg_salary,company_age,same_state,python,excel,hadoop,spark,aws,tableau,big_data
index,,,,,,,,,,,,,,,,,,,,,
0,Sr Data Scientist,137-171,Description\n\nThe Senior Data Scientist is re...,3.1,Healthfirst,"New York, NY","New York, NY",1001 to 5000 employees,1993,Nonprofit Organization,...,154,31,1,0,0,0,0,1,0,0
1,Data Scientist,137-171,"Secure our Nation, Ignite your Future\n\nJoin ...",4.2,ManTech,"Chantilly, VA","Herndon, VA",5001 to 10000 employees,1968,Company - Public,...,154,56,0,0,0,1,0,0,0,1
2,Data Scientist,137-171,Overview\n\n\nAnalysis Group is one of the lar...,3.8,Analysis Group,"Boston, MA","Boston, MA",1001 to 5000 employees,1981,Private Practice / Firm,...,154,43,1,1,1,0,0,1,0,0
3,Data Scientist,137-171,JOB DESCRIPTION:\n\nDo you have a passion for ...,3.5,INFICON,"Newton, MA","Bad Ragaz, Switzerland",501 to 1000 employees,2000,Company - Public,...,154,24,0,1,1,0,0,1,0,0
4,Data Scientist,137-171,Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions,"New York, NY","New York, NY",51 to 200 employees,1998,Company - Private,...,154,26,1,1,1,0,0,0,0,0


##### Now our data is ready for analysis, and we will save it into a csv file.

In [28]:
df.to_csv('DS_Jobs_clean.csv', index=False)